<a href="https://colab.research.google.com/github/N-Harsha-Vardhan-Dev/t5-small-linga/blob/main/Mini_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets torch


#Loading Dataset

In [5]:
from datasets import load_dataset

# Load the GECFEL dataset
dataset = load_dataset("wiki_auto", "auto_full_with_split")




README.md:   0%|          | 0.00/15.4k [00:00<?, ?B/s]

wiki_auto.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

The repository for wiki_auto contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wiki_auto.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating full split:   0%|          | 0/483801 [00:00<?, ? examples/s]

In [7]:
print(dataset)
# print(dataset['train'][0])


DatasetDict({
    full: Dataset({
        features: ['normal_sentence', 'simple_sentence'],
        num_rows: 483801
    })
})


In [13]:
len(dataset['full'])

483801

In [16]:
dataset['full'][0]
for i in range(10) :
  print(dataset['full'][i])

{'normal_sentence': 'Pterocarpus indicus ( commonly known as Amboyna wood , Malay padauk , Papua New Guinea rosewood , Philippine mahogany , Andaman redwood , Burmese rosewood , narra , angsana , or Pashu padauk ) is a species of " Pterocarpus " native to southeastern Asia , northern Australasia , and the western Pacific Ocean islands , in Cambodia , southernmost China , East Timor , Indonesia , Malaysia , Papua New Guinea , the Philippines , the Ryukyu Islands , the Solomon Islands , Thailand , and Vietnam .\n', 'simple_sentence': 'Pterocarpus indicus ( commonly known as Amboyna wood , Malay padauk , Papua New Guinea rosewood , Philippine mahogany , Andaman redwood , Burmese rosewood , narra or Pashu padauk ) is a species of " Pterocarpus " native to southeastern Asia .\n'}
{'normal_sentence': 'A romantic friendship , passionate friendship , or affectionate friendship is a very close but typically non-sexual relationship between friends , often involving a degree of physical closeness

In [17]:
from datasets import load_dataset
dataset = load_dataset("jfleg")


README.md:   0%|          | 0.00/5.94k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/141k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

In [18]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 748
    })
})

In [20]:
dataset["validation"][0]

{'sentence': 'So I think we can not live if old people could not find siences and tecnologies and they did not developped . ',
 'corrections': ['So I think we would not be alive if our ancestors did not develop sciences and technologies . ',
  'So I think we could not live if older people did not develop science and technologies . ',
  'So I think we can not live if old people could not find science and technologies and they did not develop . ',
  'So I think we can not live if old people can not find the science and technology that has not been developed . ']}

In [22]:
len(dataset['validation'])
len(dataset['test'])

748

In [29]:
from datasets import concatenate_datasets
combined_dataset = concatenate_datasets([dataset["validation"], dataset["test"]])


In [31]:
combined_dataset

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 1503
})

In [36]:
def explode_dataset(examples):
    # Split corrections into individual examples
    expanded_inputs = []
    expanded_labels = []
    for sentence, corrections in zip(examples['sentence'], examples['corrections']):
        for correction in corrections:
            expanded_inputs.append("correct: " + sentence)
            expanded_labels.append(correction)

    return {
        'sentence': expanded_inputs,
        'corrections': expanded_labels,
}

# Apply the explode function to the dataset to create one row per correction
exploded_dataset = combined_dataset.map(explode_dataset, batched=True)


Map:   0%|          | 0/1503 [00:00<?, ? examples/s]

In [38]:
exploded_dataset

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 6012
})

In [40]:
exploded_dataset[0]

{'sentence': 'correct: So I think we can not live if old people could not find siences and tecnologies and they did not developped . ',
 'corrections': 'So I think we would not be alive if our ancestors did not develop sciences and technologies . '}

In [41]:
from transformers import T5Tokenizer

# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Preprocess function for grammar correction
def preprocess_function(examples):
    # Add task prefix to the input sentence for T5 (to specify the task)
    inputs = ["correct: " + sentence for sentence in examples['sentence']]
    targets = examples['corrections']

    # Tokenize inputs and targets separately, making sure we handle them correctly
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Tokenize the target sentences
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Ensure labels are in the right format (sometimes labels might need padding token if empty)
    model_inputs["labels"] = labels["input_ids"]


    return model_inputs

# Apply the preprocessing to the combined dataset
train_dataset = exploded_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/6012 [00:00<?, ? examples/s]

In [44]:
train_dataset

Dataset({
    features: ['sentence', 'corrections', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6012
})

In [46]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [51]:
import torch
import transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Preprocessing (already done)
# train_dataset = exploded_dataset.map(preprocess_function, batched=True)

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-grammar-corrector",    # output directory
    learning_rate=3e-5,                     # learning rate
    per_device_train_batch_size=8,          # batch size for training
    per_device_eval_batch_size=8,           # batch size for evaluation
    weight_decay=0.01,                      # weight decay
    save_total_limit=3,                     # maximum checkpoints to save
    num_train_epochs=3,                     # number of training epochs
    predict_with_generate=True,             # important for seq2seq
    fp16=True if torch.cuda.is_available() else False,  # mixed precision if GPU
    # evaluation_strategy="epoch",            # evaluation strategy
    save_strategy="epoch",                  # save strategy
    report_to="none"                        # no wandb or other integrations
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # tokenized train dataset
    eval_dataset=train_dataset,   # if you don't have separate val set
    tokenizer=tokenizer,
)

# Start training
trainer.train()


<ipython-input-51-ceda739943dd>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.958200
1000,0.145500
1500,0.135100
2000,0.129300


TrainOutput(global_step=2256, training_loss=0.3177427772089099, metrics={'train_runtime': 250.4582, 'train_samples_per_second': 72.012, 'train_steps_per_second': 9.007, 'total_flos': 610256182837248.0, 'train_loss': 0.3177427772089099, 'epoch': 3.0})

#Testing

In [65]:
def correct_grammar(text: str):
    input_text = f"correct: {text}"  # Consistent with training
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=128)

    # Move to GPU if available
    input_ids = input_ids.to(model.device)

    # Generate corrected output
    output_ids = model.generate(
        input_ids,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    corrected_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return corrected_text

# Interactive testing
temp = input("Enter any sentence: ")

print(f"\nOriginal: {temp}")
corrected = correct_grammar(temp)
print(f"Corrected: {corrected}")


Enter any sentence: he is gay

Original: he is gay
Corrected: He is gay.


In [70]:
correct_grammar("You is good")

'You are good.'

In [71]:
correct_grammar("how is your day")

'How is your day?'

In [72]:
correct_grammar("how are you buddy")

'how are you?'

In [78]:
correct_grammar("how is you man")

'How is your man?'

In [79]:
correct_grammar("He go to school every day but not know much.")

'He goes to school every day but not much.'

In [81]:
correct_grammar("He go to school")

'He goes to school'

In [85]:
correct_grammar("She gone to store yesterday")

'She went to store yesterday'

#Saving model and tokenizer

In [86]:
model.save_pretrained("./t5-grammar-corrector-final")
tokenizer.save_pretrained("./t5-grammar-corrector-final")


('./t5-grammar-corrector-final/tokenizer_config.json',
 './t5-grammar-corrector-final/special_tokens_map.json',
 './t5-grammar-corrector-final/spiece.model',
 './t5-grammar-corrector-final/added_tokens.json')

In [88]:
model = T5ForConditionalGeneration.from_pretrained("./t5-grammar-corrector-final")
tokenizer = T5Tokenizer.from_pretrained("./t5-grammar-corrector-final")

In [89]:
correct_grammar("They is going to steal ")

'They are going to steal'

#Pushing to Hugging face hub

In [87]:
from huggingface_hub import notebook_login

notebook_login()


In [106]:
from huggingface_hub import create_repo, upload_folder, delete_repo


In [104]:
# 2. Create a new repository
create_repo("t5-small-updated", private=False)


RepoUrl('https://huggingface.co/Harshathemonster/t5-small-updated', endpoint='https://huggingface.co', repo_type='model', repo_id='Harshathemonster/t5-small-updated')

In [105]:
path = 't5-small-updated'
model.push_to_hub(f"Harshathemonster/{path}")
tokenizer.push_to_hub(f"Harshathemonster/{path}")

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Harshathemonster/t5-small-updated/commit/083554823790e48b8cf63d2cf15bce87fc36b315', commit_message='Upload tokenizer', commit_description='', oid='083554823790e48b8cf63d2cf15bce87fc36b315', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Harshathemonster/t5-small-updated', endpoint='https://huggingface.co', repo_type='model', repo_id='Harshathemonster/t5-small-updated'), pr_revision=None, pr_num=None)

#Importing pushed model from hugging face hub Harshathemonster

In [114]:
from transformers import T5ForTokenClassification, T5Tokenizer

def correct(text : str) :
  input = f"correct: {text}"
  input_ids = tokenizer.encode(input, return_tensors='pt', truncation=True, max_length=128)

  input_ids = input_ids.to(model.device)

  output_ids = model.generate(
      input_ids,
      max_length=128,
      num_beams=4,
      early_stopping=True
  )
  corrected_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

  return corrected_text

users_input = input("Enter any text : ")

models_output = correct(users_input)

print("\n Wrong : ", users_input)
print("\n Correct : ", models_output)


Enter any text : how is you man

 Wrong :  how is you man

 Correct :  How is your man?


In [115]:
correct("She gone to store yesterday")

'She went to store yesterday'

In [116]:
correct("He go to school every day but not know much.")

'He goes to school every day, but not much.'